collapse all those messy pattern labels into a handful of tidy analytical classes** that will plot cleanly in QGIS/Arc GIS

The idea is:

* **Pick a small, interpretable set of canonical classes** that match your research questions.
* **Write one normalization function** that turns every free-text label (regardless of typos, punctuation, or capitalization) into one of those classes.
* **Apply the function to the spreadsheet column** and save the cleaned file.

---

## Canonical classes

| Canonical name            | Typical research code | Variants/keywords it will absorb                                                                                       |
| ------------------------- | --------------------- | ---------------------------------------------------------------------------------------------------------------------- |
| **Bird and Flower**       | `BF`                  | “bird and flower”, “bird/flowers”, “bird & flower”, “peacock/pine tree”, any label containing “bird” *and* “flower”    |
| **Dragon and Phoenix**    | `DP`                  | “dragon/phoenix”, “dragon & phoenix”, “red dragon”, “phoenix”, “thunder border”, “dragon without thunder border”, etc. |
| **God of Longevity**      | `GL`                  | “god of longevity”, “longevity …”, “ATS Longevity set”, “longevity and a bat”, etc.                                    |
| **Women / Characters**    | `TW`                  | “two women”, “women/characters”, “women characters”, “characters (women)”, any label with “women” or “character”       |
| **Individualized Logo**   | `IL`                  | “individualized logo”, “custom logo”, house crest, etc.                                                                |
| **Custom / Misc Pattern** | `CP`                  | “custom pattern”, “custom flower print”, anything with “custom …”, “0”, empty cells                                    |
| **Lustre Floral**         | `LF`                  | “yellow lustre/flower”, “pink luster”, “lustre”, etc.                                                                  |
| **Other**                 | `OT`                  | Anything that slips through the nets above (kept for auditing).                                                        |


In [ ]:
from google.colab import files
import pandas as pd, io, os, re

uploaded = files.upload()
fname = next(iter(uploaded))
ext = os.path.splitext(fname)[1].lower()
raw_bytes = uploaded[fname]

if ext in {".xlsx", ".xls"}:
    df = pd.read_excel(io.BytesIO(raw_bytes))
elif ext == ".csv":
    df = pd.read_csv(io.BytesIO(raw_bytes))
else:
    raise ValueError("Please upload a .csv, .xlsx, or .xls file")

PATTERN_COL = "pattern"

LEGEND = {
    "Bird and Flower": r"bird.*flower|peacock|pine tree",
    "Dragon and Phoenix": r"dragon|phoenix|thunder border|flaming pearl|red dragon",
    "God of Longevity": r"longevity",
    "Women / Characters": r"women|character",
    "Individualized Logo": r"logo",
    "Lustre Floral": r"lustr|luster",
    "Custom / Misc Pattern": r"custom",
}

def normalize(raw):
    s = str(raw).strip().lower()
    if s in {"", "0", "nan"}:
        return "Custom / Misc Pattern"
    for canon, rx in LEGEND.items():
        if re.search(rx, s):
            return canon
    return "Other"

df[PATTERN_COL] = df[PATTERN_COL].apply(normalize)

print("\n=== Canonical counts ===")
print(df[PATTERN_COL].value_counts())

clean_name = os.path.splitext(fname)[0] + "_clean.csv"
df.to_csv(clean_name, index=False)
files.download(clean_name)
print(f"\nClean file saved & downloaded: {clean_name}")

year opened, year closed, earliest year for artifact, latest year for artifact

and produce clean, numeric “_min / _max” columns you can trust for analysis or a GIS time slider.

In [ ]:
from google.colab import files
import pandas as pd, io, os, re, numpy as np

upl      = files.upload()                     # ← choose your file
fname    = next(iter(upl))
ext      = os.path.splitext(fname)[1].lower()
b        = io.BytesIO(upl[fname])

if ext in (".xlsx", ".xls"):
    df = pd.read_excel(b)
elif ext == ".csv":
    df = pd.read_csv(b)
else:
    raise ValueError("Upload a .csv / .xlsx / .xls file")

TODAY             = pd.Timestamp.utcnow().year
STILL_OPEN_TOKEN  = ""          # put "still open" if you want to keep the text

YEAR4   = re.compile(r"\d{4}")

def decade_bounds(s):           # "195?" → (1950,1959)
    base = int(s[:3]) * 10
    return base, base + 9

def expand_short(a, b):         # "1955"-"60" → 1955-1960
    if len(b) == 2:
        b = a[:2] + b
    return int(a), int(b)

def cell_to_range(val, is_closed_col=False):
    """
    Return (min_year, max_year) from a messy cell.
    NaNs become (np.nan, np.nan).
    """
    if pd.isna(val):
        return np.nan, np.nan

    s = str(val).strip().lower()

    # Excel date‐time like "1942/3/1 00:00"
    dt = pd.to_datetime(s, errors="coerce")
    if pd.notna(dt):
        y = int(dt.year)
        return y, y

    if "still open" in s:
        return np.nan, TODAY if is_closed_col else np.nan
    if "not in" in s and "directory" in s:
        y = int(YEAR4.search(s).group())
        return y + 1, np.nan
    if ("in" in s and "directory" in s) or "by" in s or "before" in s or "≤" in s:
        y = int(YEAR4.search(s).group())
        return y, y
    if "after" in s:
        y = int(YEAR4.search(s).group())
        return y, np.nan
    if "between" in s:
        s = s.replace("between", "")

    if "til at least" in s or "to at least" in s:
        y = int(YEAR4.search(s).group())
        return np.nan, y

    m = re.fullmatch(r"(\d{4})s", s)
    if m:
        return decade_bounds(m.group(1)[:3] + "?")

    if re.fullmatch(r"\d{3}[\*\?]", s):
        return decade_bounds(s)

    m = re.fullmatch(r"(\d{4})[-/](\d{1,2})$", s)
    if m:
        return expand_short(*m.groups())

    m = re.fullmatch(r"(\d{4})[-/](\d{3,4})$", s)
    if m:
        return expand_short(*m.groups())

    m = re.fullmatch(r"(\d{4})[-/](\d{4})$", s)
    if m:
        return int(m.group(1)), int(m.group(2))

    yrs = [int(y) for y in YEAR4.findall(s)]
    if yrs:
        return min(yrs), max(yrs)

    return np.nan, np.nan

def rng_to_string(lo, hi):
    """Turn two floats into '', '1956', or '1955–1960'."""
    lo_is_nan = pd.isna(lo)
    hi_is_nan = pd.isna(hi)

    if lo_is_nan and hi_is_nan:
        return ""
    if lo_is_nan:
        return str(int(hi))        # e.g. “by 1968” ⇒ “1968”
    if hi_is_nan:
        return str(int(lo))        # e.g. “after 1955” ⇒ “1955”
    if lo == hi:
        return str(int(lo))
    return f"{int(lo)}–{int(hi)}"  # nice en-dash


YEAR_COLS = {
    "year opened":                False,
    "year closed":                True,   # special “still open” handling
    "earliest year for artifact": False,
    "latest year for artifact":   False,
}

for col, is_closed in YEAR_COLS.items():
    if col not in df.columns:
        print(f"⚠️  {col} not found – skipped")
        continue

    cleaned = []
    for val in df[col]:
        lo, hi = cell_to_range(val, is_closed_col=is_closed)
        if is_closed and isinstance(val, str) and "still open" in val.lower() and STILL_OPEN_TOKEN:
            cleaned.append(STILL_OPEN_TOKEN)
        else:
            cleaned.append(rng_to_string(lo, hi))

    df[col] = cleaned


display(df[[c for c in YEAR_COLS]])   # peek first few rows

out = os.path.splitext(fname)[0] + "_dates_clean.csv"
df.to_csv(out, index=False)
files.download(out)
print(" cleaned file downloaded:", out)


## Canonical vessel classes

| Canonical label | Catches any cell that contains …                               |
| --------------- | -------------------------------------------------------------- |
| **Tea Cup**     | `tea cup`, `teacup`                                            |
| **Saucer**      | `saucer`                                                       |
| **Ash Tray**    | `ashtray`, `ash tray`                                          |
| **Bowl**        | `bowl`, `rice/soup bowl`, `tea bowl`, `small bowl`, `wor bowl` |
| **Plate**       | `plate` (round / octagonal / raised)                           |
| **Platter**     | `platter`                                                      |
| **Dish**        | `dish` (vegetable, butter/soy, etc.)                           |
| **Tea Pot**     | `tea pot` (short spout, vertical body, thin, …)                |
| **Spoon**       | `spoon`                                                        |
| **Vase**        | `vase`                                                         |
| **Other**       | any entry that slips through the cracks                        |

In [ ]:
from google.colab import files
import pandas as pd, io, os, re

uploaded = files.upload()
fname = next(iter(uploaded))
ext = os.path.splitext(fname)[1].lower()
raw_bytes = uploaded[fname]

if ext in (".xlsx", ".xls"):
    df = pd.read_excel(io.BytesIO(raw_bytes))
elif ext == ".csv":
    df = pd.read_csv(io.BytesIO(raw_bytes))
else:
    raise ValueError("Please upload a .csv, .xlsx, or .xls file")

print(f"✅ Loaded {fname}  →  {df.shape[0]} rows")

VESSEL_KEYWORDS = {
    "Tea Cup" : ["tea cup", "teacup"],
    "Saucer"  : ["saucer"],
    "Ash Tray": ["ashtray", "ash tray"],
    "Bowl"    : ["bowl"],
    "Plate"   : ["plate", "platter"],
    "Dish"    : ["dish"],
    "Tea Pot" : ["tea pot"],
    "Spoon"   : ["spoon"],
    "Vase"    : ["vase"],
}

VESSEL_LEGEND = {
    label: re.compile("|".join(map(re.escape, words)))
    for label, words in VESSEL_KEYWORDS.items()
}

def normalize_vessel(val):
    if pd.isna(val):
        return ""
    s = str(val).strip().lower()
    for label, rx in VESSEL_LEGEND.items():
        if rx.search(s):
            return label
    return "Other"

COL = "Vessel form"
if COL not in df.columns:
    raise KeyError(f'Column "{COL}" not found – check the header name.')

df[COL] = df[COL].apply(normalize_vessel)

print("\n📝 Cleaned Vessel forms:")
print(df[COL].value_counts(dropna=False))

out_name = os.path.splitext(fname)[0] + "_vessel_clean.csv"
df.to_csv(out_name, index=False)
files.download(out_name)
print(f"\n📥 Downloaded cleaned file: {out_name}")

verifying the 'is this 1960 catagolue' column

In [ ]:
import pandas as pd, io, os, re
from google.colab import files

print("⬆️  Select the FULL dataset (Ceramic analysis FS Louie …)")
full_bytes = files.upload()
full_name = next(iter(full_bytes))
full_df = pd.read_excel(io.BytesIO(full_bytes[full_name])) \
          if full_name.endswith((".xlsx", ".xls")) else \
          pd.read_csv(io.BytesIO(full_bytes[full_name]))
print(f"✅ Loaded FULL sheet: {full_df.shape[0]} rows")

print("\n⬆️  Now select the 1960 catalog file")
cat_bytes = files.upload()
cat_name = next(iter(cat_bytes))
cat_df = pd.read_excel(io.BytesIO(cat_bytes[cat_name])) \
         if cat_name.endswith((".xlsx", ".xls")) else \
         pd.read_csv(io.BytesIO(cat_bytes[cat_name]))
print(f"✅ Loaded catalog: {cat_df.shape[0]} rows")

FULL_REST_COL = "Restuarant Name"
FULL_FLAG_COL = "Is it in the 1960 Catalog?"
CAT_REST_COL = "name"

for col in [FULL_REST_COL, FULL_FLAG_COL]:
    if col not in full_df.columns:
        raise KeyError(f'"{col}" missing in full dataset')
if CAT_REST_COL not in cat_df.columns:
    raise KeyError(f'"{CAT_REST_COL}" missing in catalog sheet')

catalog_set = set(
    cat_df[CAT_REST_COL].astype(str).str.strip().str.lower().dropna().unique()
)
print(f"🔍 Catalog has {len(catalog_set)} unique names")

def flag_match(name, cur):
    if str(cur).strip().lower() == "yes":
        return "yes"
    if pd.isna(name):
        return cur
    return "yes" if str(name).strip().lower() in catalog_set else cur

full_df[FULL_FLAG_COL] = [
    flag_match(n, f) for n, f in zip(full_df[FULL_REST_COL], full_df[FULL_FLAG_COL])
]

print("\n🏷 Flag counts after update:")
print(full_df[FULL_FLAG_COL].value_counts(dropna=False))

out_name = os.path.splitext(full_name)[0] + "_1960flag.csv"
full_df.to_csv(out_name, index=False)
files.download(out_name)
print(f"\n📥 Downloaded: {out_name}")